# ARIMAモデルのインタラクティブ・シミュレーション

本ノートブックでは、**ARIMA(p, d, q) モデル** の挙動をインタラクティブに観察します。

自己回帰項 (AR)、和分項 (I: 差分)、移動平均項 (MA) のパラメータを変更することで、
生成される時系列データや、自己相関関数 (ACF)、偏自己相関関数 (PACF) がどのように変化するかを確認してください。

## 観察のポイント
1. **AR(p) の挙動**:
   - AR係数（phi）を大きくすると、過去の値への依存が強まります。
   - ACFは徐々に減衰し、PACFはラグ $p$ でカットオフ（切断）します。
2. **MA(q) の挙動**:
   - MA係数（theta）を変更すると、過去の誤差の影響が変わります。
   - PACFは徐々に減衰し、ACFはラグ $q$ でカットオフします。
3. **非定常性 (d)**:
   - 差分階数 $d$ を1以上にすると、データはトレンドを持ち、非定常になります（ランダムウォークなど）。
   - $d=0$ の場合、データは平均の周りを振動する定常過程になります（係数が定常条件を満たす場合）。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import ipywidgets as widgets
from IPython.display import display

# 日本語フォントの設定
import matplotlib.font_manager as fm
font_path = '../fonts/ipaexg.ttf'
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'IPAexGothic'

%matplotlib inline

def plot_arima_interactive(ar1, ar2, ma1, ma2, d, n_samples=200):
    # AR係数とMA係数の設定
    # statsmodelsでは、AR項は左辺に移項した形 (1 - phi*L)y = ... なので符号を反転させる必要がある
    # しかし、ユーザーの直感的には y_t = phi * y_{t-1} が自然なので、
    # ここでは入力された正の係数を statsmodels 用に負に変換する。
    ar_params = np.array([1, -ar1, -ar2])
    ma_params = np.array([1, ma1, ma2])
    
    # ARMAプロセスの生成
    # ar1, ar2 が大きすぎると非定常になる可能性があるが、ここではそのまま通す
    try:
        process = ArmaProcess(ar_params, ma_params)
        sample = process.generate_sample(nsample=n_samples)
        
        # 差分 (Integration) の適用
        if d > 0:
            # 累積和をとることで積分過程にする（簡易的な実装）
            for _ in range(d):
                sample = np.cumsum(sample)
                
        # プロット
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # 1. 時系列プロット
        axes[0].plot(sample)
        axes[0].set_title(f"時系列データ (AR=[{ar1}, {ar2}], MA=[{ma1}, {ma2}], d={d})")
        axes[0].set_xlabel("Time")
        axes[0].set_ylabel("Value")
        axes[0].grid(True)
        
        # 定常性のチェック（AR部分の根が単位円外にあるか）
        if not process.isstationary and d == 0:
             axes[0].text(0.05, 0.9, "※非定常 (AR係数が大)", transform=axes[0].transAxes, color="red", fontweight="bold")

        # 2. 自己相関 (ACF)
        # 非定常データでACFを計算すると減衰が遅いが、ここではそのまま表示する
        plot_acf(sample, lags=20, ax=axes[1], title="自己相関 (ACF)")
        axes[1].grid(True)

        # 3. 偏自己相関 (PACF)
        try:
            plot_pacf(sample, lags=20, ax=axes[2], title="偏自己相関 (PACF)", method='ywm')
        except:
            axes[2].text(0.5, 0.5, "PACF計算エラー\n(定常性なし)", ha='center')
        axes[2].grid(True)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error: {e}")

# ウィジェットの作成
style = {'description_width': 'initial'}

w_ar1 = widgets.FloatSlider(value=0.5, min=-1.5, max=1.5, step=0.1, description='AR(1) phi1:', style=style)
w_ar2 = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='AR(2) phi2:', style=style)
w_ma1 = widgets.FloatSlider(value=0.0, min=-1.5, max=1.5, step=0.1, description='MA(1) theta1:', style=style)
w_ma2 = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='MA(2) theta2:', style=style)
w_d = widgets.IntSlider(value=0, min=0, max=2, description='差分 d:', style=style)

ui = widgets.VBox([
    widgets.HBox([w_ar1, w_ar2]),
    widgets.HBox([w_ma1, w_ma2]),
    w_d
])

out = widgets.interactive_output(plot_arima_interactive, 
                                 {'ar1': w_ar1, 'ar2': w_ar2, 
                                  'ma1': w_ma1, 'ma2': w_ma2, 
                                  'd': w_d})

display(ui, out)

## AR(1)モデルと回帰分析の視覚的理解

AR(1)モデル $y_t = \phi y_{t-1} + \epsilon_t$ は、**「現在の値 $y_t$ を、1つ前の値 $y_{t-1}$ で回帰する（説明する）」** モデルです。
これを視覚的に確認するために、時系列データのプロットと同時に、横軸に $y_{t-1}$、縦軸に $y_t$ をとった**散布図（Lag Plot）**を表示します。

AR係数 $\phi$ を変化させると、散布図上の点の分布の「傾き」や相関の強さがどのように変わるか観察してください。

In [ ]:
from sklearn.linear_model import LinearRegression

def plot_ar1_regression(phi, n_samples=200):
    # AR(1)データの生成
    # y_t = phi * y_{t-1} + epsilon
    # statsmodelsのArmaProcessを使うと符号反転が必要だが、
    # ここではシンプルに直接生成する（初期値の影響などを排除しやすい）
    np.random.seed(42)
    epsilon = np.random.normal(0, 1, n_samples)
    y = np.zeros(n_samples)
    
    # 定常分布からのサンプリングを初期値とする（定常の場合）
    if abs(phi) < 1:
        y[0] = np.random.normal(0, 1 / np.sqrt(1 - phi**2))
    else:
        y[0] = 0

    for t in range(1, n_samples):
        y[t] = phi * y[t-1] + epsilon[t]
        
    # ラグデータの作成
    y_current = y[1:]
    y_lag1 = y[:-1]
    
    # 線形回帰（確認用）
    lr = LinearRegression()
    lr.fit(y_lag1.reshape(-1, 1), y_current)
    y_pred = lr.predict(y_lag1.reshape(-1, 1))
    
    # プロット
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 1. 時系列プロット
    axes[0].plot(y)
    axes[0].set_title(f"AR(1) 時系列 (phi={phi})")
    axes[0].set_xlabel("Time")
    axes[0].set_ylabel("Value")
    axes[0].grid(True)
    
    # 2. 散布図 (Lag Plot)
    axes[1].scatter(y_lag1, y_current, alpha=0.5, label="Data")
    axes[1].plot(y_lag1, y_pred, color='red', linewidth=2, label=f"Regression Line (Slope={lr.coef_[0]:.2f})")
    axes[1].set_title(f"散布図: $y_{{t-1}}$ vs $y_t$")
    axes[1].set_xlabel("$y_{t-1}$ (1期前の値)")
    axes[1].set_ylabel("$y_t$ (現在の値)")
    axes[1].legend()
    axes[1].grid(True)
    
    # 補助線
    min_val = min(y.min(), y_lag1.min())
    max_val = max(y.max(), y_lag1.max())
    axes[1].plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--', alpha=0.5, label="y=x")

    plt.tight_layout()
    plt.show()

# ウィジェット
w_phi_reg = widgets.FloatSlider(value=0.7, min=-1.1, max=1.1, step=0.1, description='phi:', style=style)

display(widgets.interactive(plot_ar1_regression, phi=w_phi_reg))

## 任意次数のAR(p)モデルとPACFの関係

ここでは、より高次の AR($p$) モデルにおける、偏自己相関関数 (PACF) の挙動を確認します。
理論的には、AR($p$) 過程の PACF は、ラグ $p$ までは有意な値を持ち、ラグ $p+1$ 以降はゼロになります（カットオフ）。

以下のスライダーで次数 $p$ を変更し、PACFがどのように変化するかを観察してください。

※ 係数は、定常性を満たすように内部で自動的に設定されます（減衰パターンなど）。

In [ ]:
def plot_ar_p_interactive(p, n_samples=500):
    # 係数の自動設定（簡易的）
    # phi_k = 0.6 * (0.8)^(k-1) * (-1)^(k-1) みたいな減衰振動パターンを与える
    # これにより、そこそこ定常性を保ちつつ、高次まで係数がある状態を作る
    
    if p == 0:
        ar_coeffs = []
    else:
        # 少し減衰させつつ、符号を交互にするなどして特徴を出す
        # 係数の絶対値の和が1を超えると非定常になりやすいので調整
        # ここでは単調減衰パターンを採用
        signs = np.ones(p)
        # signs[1::2] = -1 # 符号交互の場合
        
        # 減衰係数
        decay = 0.6
        ar_coeffs = [decay**(i+1) * signs[i] for i in range(p)]
        
        # 係数の総和チェック（簡易的な定常性確保）
        if np.sum(np.abs(ar_coeffs)) >= 1.0:
             ar_coeffs = np.array(ar_coeffs) / (np.sum(np.abs(ar_coeffs)) + 0.1)

    # statsmodels用のARパラメータ (1, -phi1, -phi2, ...)
    ar_params = np.r_[1, -np.array(ar_coeffs)]
    ma_params = np.array([1])
    
    try:
        process = ArmaProcess(ar_params, ma_params)
        sample = process.generate_sample(nsample=n_samples)
        
        # 係数の表示
        print(f"設定されたAR係数 (phi_1 ... phi_{p}):")
        print([f"{c:.3f}" for c in ar_coeffs])
        
        if not process.isstationary:
            print("※ 注意: 生成されたプロセスは非定常の可能性があります。")

        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # 1. 時系列
        axes[0].plot(sample)
        axes[0].set_title(f"AR({p}) 時系列")
        axes[0].set_xlabel("Time")
        axes[0].grid(True)

        # 2. ACF
        plot_acf(sample, lags=20, ax=axes[1], title="自己相関 (ACF)")
        axes[1].grid(True)

        # 3. PACF
        plot_pacf(sample, lags=20, ax=axes[2], title="偏自己相関 (PACF)", method='ywm')
        # カットオフの期待位置を表示
        if p > 0:
            axes[2].axvline(x=p, color='red', linestyle='--', alpha=0.5, label=f'Lag {p}')
            axes[2].legend()
        axes[2].grid(True)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error: {e}")

# ウィジェット
w_p = widgets.IntSlider(value=1, min=0, max=10, step=1, description='次数 p:', style=style)

display(widgets.interactive(plot_ar_p_interactive, p=w_p))

## ARモデルとMAモデルの比較（ACF vs PACF）

ARモデルとMAモデル（移動平均モデル）は、**ACFとPACFのどちらが「切断（Cut off）」するか**という点で対照的な性質を持ちます。

- **AR(1)**: PACFがラグ1で切断、ACFは減衰
- **MA(1)**: ACFがラグ1で切断、PACFは減衰

以下のインタラクティブ・グラフで、モデルを切り替えてその対照性を確認してください。


In [ ]:
def plot_arma_comparison(model_type, param1):
    n_samples = 500
    
    if model_type == 'AR(1)':
        # AR(1): (1 - phi*L)y = e  => ar_params = [1, -phi]
        phi = param1
        ar_params = np.array([1, -phi])
        ma_params = np.array([1])
        title_text = rf"AR(1) with $\phi={phi:.1f}$"
        desc_text = "AR(1): ACFは減衰 (Tail off), **PACFはラグ1で切断 (Cut off)**"
        
    elif model_type == 'MA(1)':
        # MA(1): y = (1 + theta*L)e => ma_params = [1, theta]
        theta = param1
        ar_params = np.array([1])
        ma_params = np.array([1, theta])
        title_text = rf"MA(1) with $\theta={theta:.1f}$"
        desc_text = "MA(1): **ACFはラグ1で切断 (Cut off)**, PACFは減衰 (Tail off)"
        
    # Generate
    try:
        process = ArmaProcess(ar_params, ma_params)
        if not process.isstationary and model_type == 'AR(1)':
             # ARが非定常の場合
             pass
             
        sample = process.generate_sample(nsample=n_samples)
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        # ACF
        plot_acf(sample, lags=20, ax=axes[0], title="自己相関 (ACF)")
        axes[0].grid(True, alpha=0.3)
        if model_type == 'MA(1)' and abs(param1) < 1:
             axes[0].axvline(x=1, color='red', linestyle='--', alpha=0.5, label='Cut off')
             axes[0].legend()

        # PACF
        plot_pacf(sample, lags=20, ax=axes[1], title="偏自己相関 (PACF)", method='ywm')
        axes[1].grid(True, alpha=0.3)
        if model_type == 'AR(1)' and abs(param1) < 1:
             axes[1].axvline(x=1, color='red', linestyle='--', alpha=0.5, label='Cut off')
             axes[1].legend()
             
        plt.suptitle(title_text + "\n" + desc_text, fontsize=14)
        plt.tight_layout()
        display(fig)
        plt.close(fig)
        
    except Exception as e:
        print(f"Error: {e}")

# Widget
style = {'description_width': 'initial'}
w_type = widgets.ToggleButtons(options=['AR(1)', 'MA(1)'], description='Model:', value='AR(1)', style=style)
w_param = widgets.FloatSlider(value=0.7, min=-0.9, max=0.9, step=0.1, description='Coeff:', style=style)

display(widgets.interactive(plot_arma_comparison, model_type=w_type, param1=w_param))

# コラム：ARIMAモデルとは

*(夜間、街の明かりに照らされた近代的な都市で、オンライン設備を見ている若いビジネスウーマン)*

### 共同執筆者
**Joshua Noble** (Data Scientist)

## ARIMAモデルの概要
ARIMAは自己回帰和分移動平均（AutoRegressive Integrated Moving Average）の略称で、時系列分析および時系列で将来生じ得る値を予測するための手法です。

自己回帰モデリングと移動平均モデリングは、時系列データを予測するための2つの異なるアプローチです。ARIMAはこれら2つのアプローチを統合しているため、この名前が付けられています。予測は、時系列の過去の動作を使用して、その時系列の1つ以上の将来の値を予測する機械学習の一分野です。小さな店でアイスクリームを買うことを想像してみてください。天候の温暖化に伴い、アイスクリームの売上が着実に増加していることをご存じであれば、おそらく来週の注文は、この数週間の注文よりも少し増えるはずだと予測できるでしょう。どの程度大きくなるかは、今週の売上が前週の売上とどれだけ異なるかによって決まります。比較する過去がなければ未来を予測することはできないため、過去の時系列データはARIMAばかりではなく、すべての予測手法や時系列分析手法にとって非常に重要です。

ARIMA は、時系列予測に最も広く使用されているアプローチの1つであり、作業する時系列データのタイプに応じて異なる2つの方法で使用できます。最初のケースでは、時系列データの季節性を考慮する必要のない非季節性ARIMAモデルを作成しました。過去のデータのパターンに基づいて未来を予測します。2番目のケースでは、時系列に影響する規則的なサイクルである季節性を考慮しています。これらのサイクルは日次、週次、月次のいずれかに指定でき、将来の値を予測するために使用できる時系列の過去データのパターンを定義するのに役立ちます。データサイエンスの多くと同様に、予測の基礎となるのは、モデルをトレーニングするための優れた時系列データを持つことです。時系列は、変数を等間隔で測定する順序付けされた一連の値です。この等間隔の時間間隔要件のため、時間要素を含むすべてのデータセットが実際には時系列データであるとは限らないことを覚えておくことが重要です。

## ボックス・ジェンキンス法
1970年、統計学者のジョージ・ボックスとグウィリム・ジェンキンスは、あらゆる種類の時系列モデルを適合させるために、ボックス・ジェンキンス法として知られるようになった方法を提案しました。このアプローチは、時系列を生成したプロセスが定常である場合、モデルを使用して近似できるという仮定から始まります。これは、次の4つのステップで構成されています。

1. **識別**：時系列が定常であるかどうかを評価し、定常でない場合は、定常とするために必要な差分の数を評価します。次に、診断プロットで使用する差分データを生成します。自己相関および部分自己相関からデータのARMAモデルのパラメーターを特定します。
2. **推定**：データを使ってモデルのパラメーター（係数）をトレーニングします。
3. **診断チェック**：利用可能なデータとの関連で適合モデルを評価し、モデルが改善される可能性のある領域をチェックします。特に、過剰適合のチェックや残差の計算が含まれます。
4. **予測**：モデルができたら、そのモデルを使用して値の予測を開始できます。

モデルがデータに正しく適合していることを確認したら、ARIMAによる予測を開始する準備が整います。これらの各ステップを詳しく見ていきます。

## 時系列データの特徴
時系列は定常時系列か非定常時系列のいずれの場合もあります。定常時系列には、時間の経過とともに一定の統計的特性があります。つまり、平均、分散、自己相関などの統計は、データによって変化しないという意味です。ARIMAなどのほとんどの統計的予測手法は、時系列が1つ以上の変換を通じてほぼ定常にできるという仮定に基づいています。定常時系列は、統計的性質が将来も過去とほぼ同じであることを単純に予測できるため、予測が比較的容易です。非固定データの操作は可能ですが、ARIMAのようなアプローチでは困難です。

時系列データの別の主な特徴は、データにトレンドが存在するかどうかです。例えば、過去50年間に食料品店で販売されていた基本食料品の価格は、インフレによって価格が上昇するため、トレンドを示すことになります。トレンドを含むデータの予測は、トレンドによってデータ内の他のパターンが不明瞭になるため、難しい場合があります。データに安定したトレンド・ラインがあり、それが一貫して回帰する場合、それはトレンド定常である可能性があります。この場合、モデルを当てはめる前に、トレンド・ラインを当てはめ、データからトレンドを差し引くだけで、トレンドを取り除くことができます。データがトレンド定常でない場合は、差分定常である可能性があり、その場合は差分化によってトレンドを取り除くことができます。差分化を行う最も簡単な方法は、各値から前の値を引いて、時系列データにどれだけの変化があるかを測定することです。例えば、$Y_t$が期間$t$における時系列Yの値である場合、期間$t$におけるYの最初の差は$Y_t - Y_{t-1}$に等しくなります。

こちらは、非定常時系列のプロットです。それは明らかな上昇傾向があり、季節性を示しています。

*(徐々に増加する時系列を示す時系列グラフ)*

ここでの季節性は、通常の12か月サイクルです。これは、時系列を12単位ずつ差分して対処すると1990年4月と1989年4月を区別できます。12単位の遅延を伴う差分を時系列に適用すると、より定常な時系列が得られます。この時系列の差異はまだ変化していますが、ARIMAモデルはこの時系列に適合し、それを使用した予測を行うことができます。

*(前のデータを差分で示し、増加傾向を除外した時系列グラフ)*

例えば、周期的な振る舞いはあるがトレンドや季節性のない時系列も定常的です。系列を観察する際にサイクルの長さが一定でない限り、サイクルのピークと谷がどこで発生するのかを知ることはできません。一般に、定常的な時系列には、長期的には予測可能なパターンはありません。時系列データを折れ線グラフにプロットすると、一定の分散があり、大きなスパイクや減少がないため、ほぼ水平方向に見えます。

## 自己相関
自己相関を計算することで、時系列が過去の値とどの程度相関しているかを知ることができます。自己相関を計算することで、データがランダム性を示しているかどうか、ある観測値がすぐ隣の観測値とどの程度関連しているかについての質問に答えることができます。これにより、どのようなモデルがデータを最もよく表しているかがわかります。自己相関は、ラグ単位までの点間の相関関係を確認するためにプロットされることがよくあります。

自己相関における各ラグ$k$は、次のように定義されます。

$$
r_k = \frac{\sum_{t=k+1}^T (y_t - \bar{y})(y_{t-k} - \bar{y})}{\sum_{t=1}^T (y_t - \bar{y})^2}
$$

$r$は自己相関の遅れ、$T$は時系列の長さ、$y$は時系列の値を表します。自己相関係数は自己相関関数（ACF）を構成します。

ACFでは、相関係数はx軸にあり、遅れの数（遅れ順序と呼ばれる）はy軸に示されます。自己相関プロットは、statsmodelsライブラリーの`plot_acf`を使用してPythonで作成でき、`acf`関数を使用してRで作成できます。

*(自己相関関数のプロット)*

このACFプロットでは、12の時間単位のラグで差分された時系列のラグは、ゼロラグ自体と完全に相関しています。最初のラグは負、2番目のラグはわずかに正、3番目のラグは負のようになります。12番目のラグはそれ自体と強く相関していることに気付くでしょう。月次データを見ていたので、これは理にかなっています。自己相関は時系列全体でほぼ同じサイクルを維持しており、時系列にはまだ大きな季節性が含まれていることを示しています。ACFプロットは、このデータに最も適合するARIMAモデルのパラメーターを推測するのにも役立ちます。



### 自己相関の直感的理解
想像してみてください。ある日の売上や気温、株価などの時系列データがあるとします。「今日の売上」と「昨日の売上」は似たような傾向があるかもしれません。もっと前の「1週間前の売上」も、何らかの関係を持っているかもしれません。自己相関とは、この「時刻差（ラグ：lag）＝k」のときに、いまの値と1つ前、2つ前……というずらし方での相関を数値で表したものです。

自己相関を調べると、「データがどのくらい過去の値に影響されているか」が見えます。
- 時系列がランダムすぎて自己相関がほとんどない場合：過去の値から未来を予測しづらい
- 自己相関が強く残っている場合：少し前のデータから未来を予想できそう
といった判断に使えます。

## 部分自己相関関数（PACF）
時系列データにARIMAモデルを使用する準備におけるもう1つの重要なプロットは、部分自己相関関数です。ACFプロットは、$y_t$と$y_{t-k}$の間の関係を、$k$のさまざまな値について示します。$y_t$と$y_{t-1}$が相関している場合、$y_{t-1}$と$y_{t-2}$も相関します。しかし、$y_{t-2}$に含まれる新しい情報が$y_t$の予測に使われるのではなく、むしろ$y_{t-1}$と$y_{t-2}$の両方が関連しているために、$y_t$と$y_{t-2}$が相関している可能性もあります。この問題を克服するためには、偏自己相関を使用して、多くのラグ観測値を取り除くことができます。これらは、ラグ1から$k$の影響を取り除いた後の$y_t$と$y_{t-k}$の関係を測定します。それで、最初の偏自己相関は最初の自己相関と同じです。それは、除去すべき何かがないからです。各部分的な自己相関は、自己回帰モデルの最後の係数として推定できます。

RやPython、その他のプログラミング言語やライブラリーで作業していても、PACFを計算し、簡単に検査できるPACFプロットを作成する方法があります。自己相関プロットは、pythonではstatsmodelsライブラリーの`plot_pacf`を使って作成でき、Rでは`pacf`関数を使って作成できます。

*(偏自己相関関数のプロット)*

このPACFは、上記のACFプロットと同じデータを使用しています。PACFプロットは、ACFプロットのように0ではなく1から始まり、前年の同じ月と相関する1.0の遅れまで強い相関を示します。その最初の1年を過ぎると、ラグの数が増えるにつれて自己相関の量が減少していきます。年ごとに変動する月次データを見ていたので、これは理にかなっています。



### 偏自己相関の「偏」とは？
偏自己相関は、自己相関のうち、間にある他の時点の影響を取り除いたうえで、どれくらい相関があるかを見るものです。
たとえば lag=3 の自己相関を見るとき、直接「3つ前のデータ」と現在のデータを比べていますが、そこには「1つ前」「2つ前」のデータの影響も含まれているかもしれません。偏自己相関は、その中間にある影響（ラグ1・ラグ2の影響）をいったん取り除いたうえで、本当に「3つ前のデータ」と「現在のデータ」だけの純粋な相関を測ります。

### ARモデルとMAモデルの見分け方
ACFとPACFのプロットの形状は、ARIMAモデルの次数（p, q）を決定する重要な手がかりになります。

- **AR(p) モデル（自己回帰モデル）の場合**:
    - **PACF**がラグ $p$ までは有意で、ラグ $p+1$ 以降は突然ゼロ付近になります（カットオフ）。
    - ACFは徐々に減衰します。
- **MA(q) モデル（移動平均モデル）の場合**:
    - **ACF**がラグ $q$ までは有意で、ラグ $q+1$ 以降は突然ゼロ付近になります（カットオフ）。
    - PACFは徐々に減衰します。

つまり、ACFとPACFのどちらが「スパッと切れる（カットオフする）」かを見比べることで、ARモデルかMAモデルかの当たりをつけることができます。

### 観点の棚卸し（AR/MA/ARIMAの識別）

| モデル | 観点1: 自己相関 (ACF) | 観点2: 偏自己相関 (PACF) |
|---|---|---|
| **AR(p) 自己回帰** | 徐々に減衰 (Tail off) | **ラグpで切断 (Cut off)** |
| **MA(q) 移動平均** | **ラグqで切断 (Cut off)** | 徐々に減衰 (Tail off) |
| **ARMA / ARIMA** | 徐々に減衰 | 徐々に減衰 |


## 自己回帰と移動平均
その名前が示すように、ARIMAという頭字語では、渡されたパラメーターに応じて、自己回帰モデルと移動平均モデルを単一のモデルに統合します。時系列全体を通じて変化をモデリングするこれら2つの方法は関連していますが、いくつかの大きな違いがあります。自己回帰モデルでは、変数の過去値の線形結合を使用して予測します。自己回帰という用語は、変数自体に対する回帰であることを示します。この手法は、過去の値を回帰のインプットとして使用する方法が線形回帰モデルと似ています。自己回帰は次のように定義されます。

$$
y_t = c + \phi_1 y_{t-1} + \phi_2 y_{t-2} + \dots + \phi_p y_{t-p} + \varepsilon_t
$$

ここでは、$\varepsilon_t$はホワイトノイズです。これは重回帰に似ていますが、予測変数として$y_t$の遅延値を使用します。これをAR(p)モデル、つまりp次の自己回帰モデルと呼びます。

一方、移動平均モデルは、回帰で予測変数の過去の値を使用するのではなく、過去の予測誤差を使用します。移動平均は、ウィンドウ内の$k$個の値を単純に平均化し、ここで、$k$は移動平均ウィンドウのサイズであり、このウィンドウを進めます。予測値は実際の値を使用して評価され、時系列の各ステップにおける誤差が特定されます。移動平均は次のように定義されます。

$$
y_t = c + \varepsilon_t + \theta_1 \varepsilon_{t-1} + \theta_2 \varepsilon_{t-2} + \dots + \theta_q \varepsilon_{t-q}
$$

$\varepsilon_t$ はホワイトノイズです。これをMA(q)モデル、つまりq次の移動平均モデルと呼びます。もちろん、 $\varepsilon_t$の値は観測しないので、通常の意味での回帰ではありません。$y_t$の各値は、過去いくつかの予測誤差の重み付けされた移動平均と考えることができることに注意してください。

通常、ARIMAモデルでは、自己回帰項（AR）項または移動平均項（MA）のいずれかを使用します。ACFプロットとPACFプロットは、これらの用語のうちどれが最も適切かを判断するためによく使用されます。

## ARIMAモデルの指定
時系列を定常化し、自己相関の性質が判別されると、ARIMAモデルを適合させることが可能になります。ARIMAモデルには、通常、$p$、$d$、$q$と呼ばれる3つの主要なパラメーターがあります。

- **p**：ARIMAの自己回帰部分の次数
- **d**：関係する差分の程度
- **q**：移動平均部分の次数

これらは通常、ARIMA(p, d, q)の順序で書かれます。多くのプログラミング言語やパッケージは、分析する時系列とこれら3つのパラメータを使用して呼び出すことができるARIMA関数を提供しています。ほとんどの場合、データはトレーニング・セットとテスト・セットに分割され、トレーニング後にモデルの精度をテストできます。通常、時間プロットを見だけで、データに最も適切なpとqの値を知ることはできません。ただし、ACFおよびPACFプロットを使用してpとqの適切な値を決定できることが多いため、これらのプロットはARIMAを扱う上で重要な用語です。

モデルでAR用語を使用する大まかな基準は次のとおりです。

- ACFプロットは自己相関がゼロに向かって減少していることを示す
- PACFプロットはゼロに向かって急速に途切れる
- 定常系列のACFはLag - 1で正を示す

モデルでMAの用語を使用する状況のおおよその基準は、次の場合です。

- ラグ-1で負に自己相関
- 数回の遅れで急激に低下するACF
- PACFが突然ではなく徐々に減少

遭遇する可能性のある従来のARIMAモデル・タイプがいくつかあります。

- **ARIMA (1,0,0) = 1次自己回帰モデル**: 系列が定常で自己相関関係にある場合、おそらくそれ自体の以前の値の倍数に定数を加えたものとして予測できます。今日からのアイスクリームの売上だけを使って明日のアイスクリームの売上を直接予測できるとしたら、それは一次自己回帰モデルです。
- **ARIMA(0,1,0) = ランダム・ウォーク**: 時系列が定常でない場合、最も単純なモデルはランダム・ウォーク・モデルです。ランダム・ウォークは、シーケンス内の次の値がシーケンス内の前の値の修正であるため、乱数のリストとは異なります。これは、よくある、株価の差異値のモデル化方法です。
- **ARIMA(1,1,0) = 差分一次自己回帰モデル**: ランダム・ウォーク・モデルのエラーが自己相関している場合、予測方程式に従属変数の1つのラグを追加することで、つまり、Yの最初の差分を1期間ラグしたそれ自体に回帰することで、問題を解決できる可能性があります。
- **定数なしのARIMA(0,1,1) = 単純な指数平滑化モデル**: これは、季節性やトレンドのない時系列データに使用されます。過去の観測からの影響度（0～1の間の係数値で示される）を制御する単一の平滑化パラメーターが必要です。この手法では、1に近い値はモデルが過去の観測値にほとんど注意を払うことを意味し、小さい値は予測中により多くの履歴を考慮することを意味します。
- **定数付きARIMA(0,1,1) = 単純な指数平滑化成長モデル**: これは、単純な指数平滑化と同じですが、時系列のY値が進行するにつれて増大する付加的な定数項がある点が異なります。

もちろん、ARIMAモデルを適合させる方法は他にもたくさんあります。そのため、複数のモデルを計算し、比較して、どのモデルがデータに最もよく適合するかを確認することがよくあります。これらはすべて一次モデルであり、線形プロセスをマッピングすることを意味します。二次プロセスをマッピングする2次モデルと、より複雑なプロセスをマッピングする上位モデルがあります。

## ARIMAモデルの比較
通常、複数のARIMAモデルをデータに適合させ、相互に比較することで、どの資産が時系列データに見られるパターンを予測しているかを見つけられます。ARIMAモデルの精度を評価するための重要な指標は次の3つです。

1. **Akaikeの情報基準またはAIC**。これは、回帰モデルの予測子の選択に広く使用されており、ARIMAモデルの次数の決定にも役立ちます。AICは、モデルの適合性とモデルの単純さないし倹約性の両方を単一の統計情報で定量化します。AICスコアが低いほど良いため、スコアが低いモデルを好みます。AICはより単純なモデルを優先し、より複雑なモデルは、その精度がより単純なモデルとほぼ同じである限り、より高いスコアを獲得します。また、単にサンプル・サイズに小さな修正を加えた修正済みAICやAICCもあります。
2. **ベイズ情報基準（BIC）**。これは、AICよりも多くの複雑さにペナルティーを与えるモデル選択のもう1つの基準です。AICと同様に、BICの低いモデルは、一般的にスコアの高いモデルよりも優先されます。モデルが長期予測に使用される場合は、BICの方が適している可能性がありますが、短期予測にはAICの方が適しているということかもしれません。
3. **シグマ二乗またはsigma2値**は、モデル残差の分散です。シグマは、仮想プロセスの不安定性を表します。揮発性が高いデータにもかかわらずシグマ二乗スコアが非常に低い場合や、逆に非揮発性データであるがシグマ二乗スコアが高い場合は、モデルが実際のデータ生成プロセスをうまく捉えていないことを示しています。

テスト・データ・セットを留保した場合は、異なる予測間隔のRMSEなどの精度メトリクスを比較することもできます。ARIMAモデルは、将来の単一の時間ステップまたは一度に複数のステップの値を予測できます。

## ARIMAのバリエーション
ARIMAモデルの構成と比較に対するもう1つのアプローチは、自動化された構成タスクをARIMAモデルの生成と比較に適用するAuto-ARIMAを使用することです。最適なモデルを得るには、複数の方法があります。アルゴリズムは複数のモデルを生成し、AICcと最尤推定の誤差を最小化してARIMAモデルを得ようとします。

**季節性自己回帰統合移動平均、SARIMAまたは季節性ARIMA**は、季節的要素を含む時系列データをサポートするARIMAの拡張版です。そのために、系列の季節成分の自己回帰、差分、移動平均を指定する3つの新しいハイパーパラメーターと、季節性の期間のパラメーターを追加します。SARIMAモデルは通常、SARIMA((p,d,q),(P,D,Q))と表現され、小文字は時系列の非季節性成分を示し、大文字は季節性成分を示します。

**ベクトル自己回帰モデル（またはVARモデル）**は、多変量の時系列に使用されます。これらは、各変数がそれ自体の過去の遅延と他の変数の過去の遅延の線形関数になるように構成されています。

ARIMAモデルは、時系列データを分析して過去のプロセスを理解したり、時系列の将来の値を予測したりするための強力なツールです。ARIMAモデルは、自己回帰モデルと移動平均モデルを組み合わせて、予測者にさまざまな時系列データを使用できる高度にパラメーター化可能なツールを提供します。
